# 1.使用Pandas整理Excel資料

In [1]:
#下載資料
import pandas as pd
df = pd.read_excel('https://www.stat.gov.tw/public/Attachment/712693030RPKUP4RX.xlsx', header=3)
df

,縣市代碼,縣市名稱,區里代碼,區鄉鎮名稱,村里代碼,村里名稱,村里代碼.1
0,63,台北市,6300100,松山區,6300100-002,莊敬里,63000010002
1,63,台北市,6300100,松山區,6300100-003,東榮里,63000010003
2,63,台北市,6300100,松山區,6300100-004,三民里,63000010004
3,63,台北市,6300100,松山區,6300100-005,新益里,63000010005
4,63,台北市,6300100,松山區,6300100-006,富錦里,63000010006
...,...,...,...,...,...,...,...
7846,9020,金門縣,902005,烈嶼鄉,0902005-003,西口村,9020050003
7847,9020,金門縣,902005,烈嶼鄉,0902005-004,林湖村,9020050004
7848,9020,金門縣,902005,烈嶼鄉,0902005-005,黃埔村,9020050005
7849,9020,金門縣,902006,烏坵鄉,0902006-001,大坵村,9020060001


In [2]:
#篩選資料
df.drop(columns=['縣市代碼', '村里代碼', '村里名稱', '村里代碼', '村里代碼.1'], axis=1, inplace=True)
df

,縣市名稱,區里代碼,區鄉鎮名稱
0,台北市,6300100,松山區
1,台北市,6300100,松山區
2,台北市,6300100,松山區
3,台北市,6300100,松山區
4,台北市,6300100,松山區
...,...,...,...
7846,金門縣,902005,烈嶼鄉
7847,金門縣,902005,烈嶼鄉
7848,金門縣,902005,烈嶼鄉
7849,金門縣,902006,烏坵鄉


In [3]:
#去除重複資料
df.drop_duplicates(inplace=True)
df

,縣市名稱,區里代碼,區鄉鎮名稱
0,台北市,6300100,松山區
33,台北市,6300200,信義區
74,台北市,6300300,大安區
127,台北市,6300400,中山區
169,台北市,6300500,中正區
...,...,...,...
7822,金門縣,902002,金沙鎮
7830,金門縣,902003,金湖鎮
7838,金門縣,902004,金寧鄉
7844,金門縣,902005,烈嶼鄉


In [4]:
#儲存資料
df.to_csv('Township.csv', encoding='big5', index=False)

# 2.查詢鄉鎮市區天氣預測

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

# 檢查是否有鄉鎮市區代碼檔
if not os.path.isfile('Township.csv'):
    df = pd.read_excel('https://www.stat.gov.tw/public/Attachment/712693030RPKUP4RX.xlsx', axis=1, inplace=True)
    df.drop(columns=['縣市代碼', '村里代碼', '村里名稱', '村里代碼', '村里代碼.1'], axis=1, inplace=True)
    df.drop_duplicates(inplace=True)
    df.to_csv('district.csv', encoding='big5', index=False)


dftown = pd.read_csv('Township.csv', encoding='big5')  #區鄉鎮名稱代碼資料
town = input('請輸入查詢的鄉鎮市區名稱：')  #要查詢的區鄉鎮名稱 
dfs = dftown[(dftown['縣市名稱']==town[:3]) & (dftown['區鄉鎮名稱']==town[3:])]
if len(dfs) > 0:  #區鄉鎮名稱存在
    town_no = str(dfs.iloc[0,1])
    url = 'https://www.cwb.gov.tw/V8/C/W/Town/MOD/3hr/' + town_no + '_3hr_PC.html'  #三日預報網頁
    res = requests.get(url)
    res.encoding = 'utf-8'
    soup = BeautifulSoup(res.text, 'lxml')
    
    df = pd.DataFrame()
    # 日期 時間
    day1 = soup.select('#PC3_D1')[0].text[:5]
    day2 = soup.select('#PC3_D2')[0].text[:5]
    day3 = soup.select('#PC3_D3')[0].text[:5]
    timeall = soup.select('.time')[0]
    tem = timeall.find_all('th')
    daylist = []
    for i in range(1, len(tem)):
        if(tem[i].get('headers')[1]) == 'PC3_D1':
            daylist.append(day1)
        elif(tem[i].get('headers')[1]) == 'PC3_D2':
            daylist.append(day2)
        else:
            daylist.append(day3)
    df['日期'] = pd.Series(daylist)
    tem = timeall.find_all('span')
    timelist = [tem1.text for tem1 in tem]
    df['時間'] = pd.Series(timelist)
    
    # 天氣示意圖
    body = soup.find_all('tr')
    wx = body[2].find_all('img')
    wxlist = [tem.get('title') for tem in wx]
    df['天氣狀況'] = pd.Series(wxlist)
    
    # 溫度
    tc = body[3].find_all('span', class_='tem-C is-active')
    tclist = [tem.text for tem in tc]
    df['溫度'] = pd.Series(tclist)
    
    # 體感溫度
    tb = body[4].find_all('span', class_='tem-C is-active')
    tblist = [tem.text for tem in tb]
    df['體感溫度'] = pd.Series(tblist)
    
    # 降雨機率 
    rain = body[5].find_all('td')
    rainlist = []
    for i in range(0, len(rain)):
        try:
            n = rain[i].get('colspan')
        except:
            n = None
        rainlist.append(rain[i].text)
        if n != None:
            rainlist.append(rain[i].text)
    df['降雨機率'] = pd.Series(rainlist)
        
    # 相對濕度
    rh = body[6].find_all('td')
    rhlist = [tem.text for tem in rh]
    df['相對濕度'] = pd.Series(rhlist)
    
    # 蒲福風級
    bf = body[7].find_all('td')
    bflist = [tem.text for tem in bf]
    df['蒲福風級'] = pd.Series(bflist)
    
    # 風向
    wd = body[9].find_all('td')
    wdlist = [tem.text for tem in wd]
    df['風向'] = pd.Series(wdlist)
    
    # 舒適度
    rh = body[10].find_all('td')
    rhlist = [tem.text for tem in rh]
    df['舒適度'] = pd.Series(rhlist)
    
    # 欄位重新排序
    df = df[['時間','日期','天氣狀況','溫度','降雨機率','體感溫度','相對濕度','舒適度','蒲福風級','風向']]
else:
    print('無此鄉鎮市區名稱！')    
    # 轉為json回傳
    #print(df.to_json(orient='records', force_ascii=False))
#else:
    #print('無此鄉鎮市區名稱！')

請輸入查詢的鄉鎮市區名稱：新北市淡水區


In [2]:
#儲存資料
df.to_csv(encoding='utf-8', index=False)
df

,時間,日期,天氣狀況,溫度,降雨機率,體感溫度,相對濕度,舒適度,蒲福風級,風向
0,21:00,04/18,多雲,23,10%,26,89%,舒適,2,東南風
1,00:00,04/19,陰,24,10%,27,86%,舒適,≤1,偏南風
2,03:00,04/19,多雲,24,10%,27,90%,舒適,2,偏南風
3,06:00,04/19,多雲,23,10%,25,87%,舒適,2,偏南風
4,09:00,04/19,陰,27,10%,30,78%,舒適,2,偏北風
5,12:00,04/19,多雲,29,30%,33,79%,悶熱,4,西南風
6,15:00,04/19,短暫陣雨或雷雨,29,30%,30,80%,舒適,4,西南風
7,18:00,04/19,短暫陣雨或雷雨,26,30%,25,85%,舒適,4,西南風
8,21:00,04/19,短暫陣雨或雷雨,24,30%,24,88%,舒適,3,西南風
9,00:00,04/20,陣雨或雷雨,24,70%,25,84%,舒適,4,西南風


# 3.使用Flask網站建置 Web API

In [ ]:
from flask import Flask
app = Flask(__name__)

import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

@app.route('/<town>')
def index(town):
    if not os.path.isfile('Township.csv'):
        df = pd.read_excel('https://www.stat.gov.tw/public/Attachment/712693030RPKUP4RX.xlsx', header=3)
        df.drop(columns=['縣市代碼', '村里代碼', '村里名稱', '村里代碼', '村里代碼.1'], axis=1, inplace=True)
        df.drop_duplicates(inplace=True)
        df.to_csv('district.csv', encoding='big5', index=False)
    
    
    dftown = pd.read_csv('Township.csv', encoding='big5')  #區鄉鎮名稱代碼資料
    town = input('請輸入查詢的鄉鎮市區名稱：')  #要查詢的區鄉鎮名稱 
    dfs = dftown[(dftown['縣市名稱']==town[:3]) & (dftown['區鄉鎮名稱']==town[3:])]
    if len(dfs) > 0:  #區鄉鎮名稱存在
        town_no = str(dfs.iloc[0,1])
        url = 'https://www.cwb.gov.tw/V8/C/W/Town/MOD/3hr/' + town_no + '_3hr_PC.html'  #三日預報網頁
        res = requests.get(url)
        res.encoding = 'utf-8'
        soup = BeautifulSoup(res.text, 'lxml')
        
        df = pd.DataFrame()
        # 日期 時間
        day1 = soup.select('#PC3_D1')[0].text[:5]
        day2 = soup.select('#PC3_D2')[0].text[:5]
        day3 = soup.select('#PC3_D3')[0].text[:5]
        timeall = soup.select('.time')[0]
        tem = timeall.find_all('th')
        daylist = []
        for i in range(1, len(tem)):
            if(tem[i].get('headers')[1]) == 'PC3_D1':
                daylist.append(day1)
            elif(tem[i].get('headers')[1]) == 'PC3_D2':
                daylist.append(day2)
            else:
                daylist.append(day3)
        df['日期'] = pd.Series(daylist)
        tem = timeall.find_all('span')
        timelist = [tem1.text for tem1 in tem]
        df['時間'] = pd.Series(timelist)
        
        # 天氣示意圖
        body = soup.find_all('tr')
        wx = body[2].find_all('img')
        wxlist = [tem.get('title') for tem in wx]
        df['天氣狀況'] = pd.Series(wxlist)
        
        # 溫度
        tc = body[3].find_all('span', class_='tem-C is-active')
        tclist = [tem.text for tem in tc]
        df['溫度'] = pd.Series(tclist)
        
        # 體感溫度
        tb = body[4].find_all('span', class_='tem-C is-active')
        tblist = [tem.text for tem in tb]
        df['體感溫度'] = pd.Series(tblist)
        
        # 降雨機率 
        rain = body[5].find_all('td')
        rainlist = []
        for i in range(0, len(rain)):
            try:
                n = rain[i].get('colspan')
            except:
                n = None
            rainlist.append(rain[i].text)
            if n != None:
                rainlist.append(rain[i].text)
        df['降雨機率'] = pd.Series(rainlist)
            
        # 相對濕度
        rh = body[6].find_all('td')
        rhlist = [tem.text for tem in rh]
        df['相對濕度'] = pd.Series(rhlist)
        
        # 蒲福風級
        bf = body[7].find_all('td')
        bflist = [tem.text for tem in bf]
        df['蒲福風級'] = pd.Series(bflist)
        
        # 風向
        wd = body[9].find_all('td')
        wdlist = [tem.text for tem in wd]
        df['風向'] = pd.Series(wdlist)
        
        # 舒適度
        rh = body[10].find_all('td')
        rhlist = [tem.text for tem in rh]
        df['舒適度'] = pd.Series(rhlist)
        
        # 欄位重新排序
        df = df[['時間','日期','天氣狀況','溫度','降雨機率','體感溫度','相對濕度','舒適度','蒲福風級','風向']]
        # 轉為return
        return df.to_json(orient='records', force_ascii=False)
    else:
        return'無此鄉鎮市區名稱！'
    
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [19/Apr/2023 20:06:21] "GET / HTTP/1.1" 404 -


請輸入查詢的鄉鎮市區名稱：台北市士林區


127.0.0.1 - - [19/Apr/2023 20:06:34] "GET /favicon.ico HTTP/1.1" 200 -


請輸入查詢的鄉鎮市區名稱：台北市士林區


127.0.0.1 - - [19/Apr/2023 20:06:44] "GET /台北市士林區 HTTP/1.1" 200 -
127.0.0.1 - - [19/Apr/2023 20:07:58] "GET / HTTP/1.1" 404 -
